<div style=" border-bottom: 8px solid #e3f56c; overflow: hidden; border-radius: 10px; height: 95%; width: 100%; display: flex;">
  <div style="height: 100%; width: 100%; background-color: #3800BB; float: left; text-align: center; display: flex; justify-content: left; align-items: center; font-size: 40px; ">
    <b><span style="color: #FFFFFF; padding: 20px 20px;">Blending</span></b>
  </div>
</div>



<div class="alert" style="background-color: #FFFFFF; border-left: 8px solid #B12111; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

<div class="alert alert-danger">

### **Contents** 
</div>

<hr>

<div class="alert">

  <p><font size="3" face="Arial" font-size="large">
  <ul type="square">

  <li> Train Catboost;  </li>
  <li> Train LightGBM;  </li>
  <li> Train XGBoost;  </li>
  <li> Blending and some of its key concepts;  </li>
  <li> Conclusions and Summary;  </li>
  
  </ul>
  </font></p>

</div>

</div>

<div class="alert alert-warning">

### **Intro** 
</div>

<div class="alert" style="background-color:#E8F8F5; border-left: 8px solid #1ABC9C; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

* text
* text
* text
* text

</div>

<div class="alert alert-success">

### **Conclusions and Summary** 
</div>

<div class="alert" style="background-color:rgb(255, 255, 255); border-left: 8px solid #5ad197; padding: 14px; border-radius: 8px; font-size: 14px; color: #000000;">

* 
* 
* 


</div>